In [1]:
import numpy as np

Les données de test:(passage de format Conllu en format text)

In [2]:

input_file = 'test.conllu'
test_file = 'test_pos.txt'

with open(input_file, 'r', encoding='utf-8') as input_f, open(test_file, 'w', encoding='utf-8') as output_f:
    for line in input_f:
        line = line.strip()
        if line and not line.startswith('#'):
            columns = line.split('\t')
            word = columns[1]  # L'indice 1 correspond à la colonne FORM (mot)
            pos = columns[3]  # L'indice 3 correspond à la colonne UPOS (partie du discours universelle)
            output_f.write(f"{word}\t{pos}\n")

Les données de train:(passage de format Conllu en format text)

In [3]:
input_file = 'train.conllu'
train_file = 'train_pos.txt'

with open(input_file, 'r', encoding='utf-8') as input_f, open(train_file, 'w', encoding='utf-8') as output_f:
    for line in input_f:
        line = line.strip()
        if line and not line.startswith('#'):
            columns = line.split('\t')
            word = columns[1]  # L'indice 1 correspond à la colonne FORM (mot)
            pos = columns[3]  # L'indice 3 correspond à la colonne UPOS (partie du discours universelle)
            output_f.write(f"{word}\t{pos}\n")

Construction de vocabulaire a partir des données de train :

In [4]:
voc_l = set()

with open('train_pos.txt', 'r', encoding='utf-8') as input_f:
    for line in input_f:
        line = line.strip()
        if line:
            word, pos = line.split('\t')
            voc_l.add(word)
voc_l=list(voc_l)

Affichage de qlq exemples de vocabulaire 

In [5]:
print("Vocabulaire:")
print("A few items of the vocabulary list")
print(voc_l[0:50])
print()
print("A few items at the end of the vocabulary list")
print(voc_l[-50:])

Vocabulaire:
A few items of the vocabulary list
['ascendant', 'campagne', 'Network', 'boucher', 'osseux', 'Philosophy', 'ḏāl', 'littéraires', 'tropicales', 'attributions', 'Albéric', 'Bannoncourt', 'Classée', 'Camposanto', 'bourguignon', 'Sahraouie', 'arriveront', 'Commissaire', 'Opterre', 'Perpétuel', 'Gordes', 'hêmisus', 'lâché', 'Carahes', 'cristallin', 'novosti', 'Meierfrankenfeld', 'acceptable', 'XML', '+0,6', 'écoulant', 'destructeurs', 'Vanuatu', 'continuent', 'authentiquement', 'retracée', 'impeccables', 'PSP', 'Ken', 'Équateur', 'Bourbons', 'auxiliaires', 'liturgique', 'Ramgoolam', 'paie', 'Sagittaire', 'decumbens', 'sensibles', 'Sincèrement', 'those']

A few items at the end of the vocabulary list
['attitude', 'Baptême', 'restantes', 'techniciens', 'Antonio', 'Capra', 'GMT', 'fausses', 'Min', 'protège', 'Insee', '-je', 'stock', 'Villemario', 'Goulag', 'regorge', 'aérage', 'Caïn', 'Schleswig-Holstein', 'Cyangugu', 'Sifuentes', 'aie', 'End', 'joie', 'démonstration', 'Boycott', 



Lire toutes les lignes du fichier train_pos en tant que liste de chaînes de caractères. 
Chaque élément de la liste correspond à une ligne du fichier.

In [6]:
# load in the training corpus
with open("train_pos.txt", 'r', encoding='utf-8') as f:
    training_corpus = f.readlines()

In [7]:
print(f"A few items of the training corpus list")
print(training_corpus[0:5])

A few items of the training corpus list
['Les\tDET\n', 'commotions\tNFP\n', 'cérébrales\tADJFP\n', 'sont\tAUX\n', 'devenu\tVPPMS\n']


Créer un dictionnaire (vocab) qui mappe chaque mot à un indice unique.
la clé est le mot et la valeur c'est son unique indice.
Il ajoute également une entrée "--unk--" pour représenter les mots inconnus. 

In [8]:
# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
vocab['--unk--'] = len(vocab)    

Compter le nombre d'apparitions de chaque mot dans une liste de phrases tokenisées et de retourner un dictionnaire qui mappe chaque mot à sa fréquence.

In [10]:
def count_words(tokenized_sentences):
        
    word_counts = {}
    
    for sentence in tokenized_sentences: 
        for token in sentence: 
            if token not in word_counts.keys(): 
                word_counts[token] = 1
            
            else:
                word_counts[token] += 1    
    return word_counts

In [11]:
# load in the test corpus
with open("test_pos.txt", 'r', encoding='utf-8') as f:
    y = f.readlines()

In [12]:
print("A sample of the test corpus")
print(y[0:10])

A sample of the test corpus
['Je\tPPER1S\n', 'sens\tVERB\n', "qu'\tCOSUB\n", 'entre\tPREP\n', 'ça\tPDEMMS\n', 'et\tCOCO\n', 'les\tDET\n', 'films\tNMP\n', 'de\tPREP\n', 'médecins\tNMP\n']


In [13]:
import string

# Caractères de ponctuation
punct = set(string.punctuation)

# Règles de morphologie utilisées pour attribuer des jetons aux mots inconnus
noun_suffix = ["age", "al", "eau", "ée", "euse", "ie", "ière", "ion", "ité", "oire", "sme", "té", "tion"]
verb_suffix = ["er", "ir", "oir", "re"]
adj_suffix = ["able", "aire", "al", "e", "el", "esque", "et", "i", "ien", "ier", "if", "in", "ique", "iste", "itif", "ive", "oire", "ol", "on", "ot", "t", "é"]
adv_suffix = ["amment", "emment", "iment"]



Extraire uniquement les mots de chaque ligne

In [14]:
with open('test_pos.txt', 'r', encoding='utf-8') as file:
    data = file.read()

# Diviser le texte en lignes
lines = data.split('\n')

# Extraire uniquement les mots de chaque ligne
prep = []
for line in lines:
    tokens = line.split('\t')
    if len(tokens) > 1:
        word = tokens[0]
        prep.extend([word])

# Afficher la liste des mots extraits
print(prep[0:20])

['Je', 'sens', "qu'", 'entre', 'ça', 'et', 'les', 'films', 'de', 'médecins', 'et', 'scientifiques', 'fous', 'que', 'nous', 'avons', 'déjà', 'vus', ',', 'nous']


Remplacer les mots de l'ensemble des donnees de teste qui ne font pas partie du vocabulaire donné par le symbole '<--unk-->' 

In [15]:
def assign_unk(word):
    return '--unk--' if word not in vocab.keys() else word
prep = [assign_unk(word) for word in prep]
print(prep[0:20])

['Je', 'sens', "qu'", 'entre', 'ça', 'et', 'les', 'films', 'de', 'médecins', 'et', 'scientifiques', 'fous', 'que', 'nous', 'avons', 'déjà', 'vus', ',', 'nous']


- Vérifier si la ligne est vide,si c'est le cas, il n'y a pas de mot ou d'étiquette associée. Dans ce cas, on attribue les valeurs --n-- au mot et --s-- l'étiquette.
- Vérifier ensuite si la longueur de split_line est différente de 2. Si c'est le cas, cela signifie que la ligne ne contient pas à la fois un mot et une étiquette valide. Dans ce cas, on attribue les valeurs --n-- au mot et --s-- à l'étiquette, et les renvoie.
- Si la longueur de split_line est égale à 2, cela signifie que la ligne contient à la fois un mot et une étiquette valide. Elle assigne le mot et l'étiquette aux variables word et tag, respectivement.

In [16]:
def get_word_tag(line, vocab):
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        split_line = line.split()
        if len(split_line) != 2:
            word = "--n--"
            tag = "--s--"
        else:
            word, tag = split_line
            if word not in vocab:
                # Gérer les mots inconnus
                word = assign_unk(word)
        return word, tag

In [17]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np

- La foction prend un corpus d'entraînement et un vocabulaire en entrée, et renvoie trois dictionnaires : emission_counts, transition_counts et tag_counts.
- Initialiser la variable prev_tag avec l'état de départ, représenté par --s-- .Cela représente l'état précédent dans la séquence de tags.
- Vérifier si i est un multiple de 50 000. Si c'est le cas, on affiche le nombre de mots traités jusqu'à présent.
- On appelle la fonction get_word_tag pour gèrer les mots inconnus et attribuer des valeurs spéciales.
- emission_counts: un dictionnaire pour calculer la proba sachant un tag donne.
- transition_counts : un dictionnaire pour calculer le nombre de fois un tag donne est apparue a cote d'un autre tag donne.
- tag_counts : le nobre de fois qu'un tag est apparue.

In [18]:
def create_dictionaries(training_corpus, vocab):

    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    i = 0 
    
    for word_tag in training_corpus:
        i += 1
        #signifie que 50000 mots ont été traités depuis le dernier affichage de compte.
        if i % 50000 == 0:
            print(f"word count = {i}")
            
        word, tag = get_word_tag(word_tag,vocab) 
        
        transition_counts[(prev_tag, tag)] += 1
        
        emission_counts[(tag, word)] += 1

        tag_counts[tag] += 1

        prev_tag = tag
        
        
    return emission_counts, transition_counts, tag_counts

In [19]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000


- Tout les tags :

In [20]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 67
View these POS tags (states)
['--s--', 'ADJ', 'ADJFP', 'ADJFS', 'ADJMP', 'ADJMS', 'ADV', 'AUX', 'CHIF', 'COCO', 'COSUB', 'DET', 'DETFS', 'DETMS', 'DINTFS', 'DINTMS', 'INTJ', 'MOTINC', 'NFP', 'NFS', 'NMP', 'NMS', 'NOUN', 'NUM', 'PART', 'PDEMFP', 'PDEMFS', 'PDEMMP', 'PDEMMS', 'PINDFP', 'PINDFS', 'PINDMP', 'PINDMS', 'PINTFS', 'PPER1S', 'PPER2S', 'PPER3FP', 'PPER3FS', 'PPER3MP', 'PPER3MS', 'PPOBJFP', 'PPOBJFS', 'PPOBJMP', 'PPOBJMS', 'PREF', 'PREFP', 'PREFS', 'PREL', 'PRELFP', 'PRELFS', 'PRELMP', 'PRELMS', 'PREP', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'VPPFP', 'VPPFS', 'VPPMP', 'VPPMS', 'VPPRE', 'X', 'XFAMIL', 'YPFOR', '_']


- Quelques exemples des dictionnaires:

In [21]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'vert': print (tup, cnt) 

transition examples: 
(('--s--', 'DET'), 1)
(('DET', 'NFP'), 4162)
(('NFP', 'ADJFP'), 1452)

emission examples: 
(('ADJMP', 'autres'), 135)
(('ADJMP', 'petits'), 29)
(('NMP', 'fruits'), 9)

ambiguous word example: 
('ADJMS', 'vert') 11
('NMS', 'vert') 2


# Phase de test :

Cette fonction utilise emission_counts et lensemble des tags pour prédire les POS des mots dans le corpus de prétraitement (prep). Elle compare ensuite ces prédictions aux étiquettes de référence pour calculer la précision du modèle de prédiction de POS.(accuracy)
matrice d emission est proba d'un mot observable sachant qu'on est dans tag donné
matrice de transition est la proba des tags sachant son contexte ( avant et lui mm)

In [22]:
def predict_pos(prep, y, emission_counts, vocab, states):

    num_correct = 0    
    all_words = set(emission_counts.keys())
    
    total = len(y)
    for word, y_tup in zip(prep, y): 

        y_tup_l = y_tup.split()        
        # Verify that y_tup contain both word and POS
        if len(y_tup_l) == 2:            
            # Set the true POS label for this word
            true_label = y_tup_l[1]

        else:
            # If the y_tup didn't contain word and POS, go to next word
            continue
    
        count_final = 0
        pos_final = ''
        
        # If the word is in the vocabulary...
        if word in vocab:
            for pos in states:

            ### START CODE HERE (Replace instances of 'None' with your code) ###
                        
                # define the key as the tuple containing the POS and word
                key = (pos,word)

                # check if the (pos, word) key exists in the emission_counts dictionary
                if key in emission_counts: # complete this line

                # get the emission count of the (pos,word) tuple 
                    count = emission_counts[key]

                    # keep track of the POS with the largest count
                    if count>count_final: # complete this line

                        # update the final count (largest count)
                        count_final = count

                        # update the final POS
                        pos_final = pos

            # If the final POS (with the largest count) matches the true POS:
            if pos_final == true_label: # complete this line
                
                # Update the number of correct predictions
                num_correct += 1
            
    ### END CODE HERE ###
    accuracy = num_correct / total
    
    return accuracy

In [23]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8775


- Hidden Markov Models for POS

- On utilise tag_counts et transition_counts pour créer une matrice de transition utilisée dans le modèle HMM pour prédire les étiquettes POS des mots. 
- La lissage est appliquée pour gérer les transitions qui n'ont pas été observées dans les données d'entraînement.(pour eviter les valeurs nulles)
- La matrice de transition regroupe les probas de transition entre les differents etats, la somme de chaue ligne egale a 1.
- les colonne et les lignes de cette matrice sont les tags.

In [24]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: create_transition_matrix
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Input: 
        alpha: number used for smoothing
        tag_counts: a dictionary mapping each tag to its respective count
        transition_counts: transition count for the previous word and tag
    Output:
        A: matrix of dimension (num_tags,num_tags)
    '''
    # Get a sorted list of unique POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Count the number of unique POS tags
    num_tags = len(all_tags)
    
    # Initialize the transition matrix 'A'
    A = np.zeros((num_tags,num_tags))
    
    # Get the unique transition tuples (previous POS, current POS)
    trans_keys = set(transition_counts.keys())
    
    ### START CODE HERE (Return instances of 'None' with your code) ### 
    
    # Go through each row of the transition matrix A
    for i in range(num_tags):
        
        # Go through each column of the transition matrix A
        for j in range(num_tags):

            # Initialize the count of the (prev POS, current POS) to zero
            count = 0
        
            # Define the tuple (prev POS, current POS)
            # Get the tag at position i and tag at position j (from the all_tags list)
            key = (all_tags[i],all_tags[j])

            # Check if the (prev POS, current POS) tuple 
            # exists in the transition counts dictionaory
            if transition_counts: #complete this line
                
                # Get count from the transition_counts dictionary 
                # for the (prev POS, current POS) tuple
                count = transition_counts[key]
                
            # Get the count of the previous tag (index position i) from tag_counts
            count_prev_tag = tag_counts[all_tags[i]]
            
            # Apply smoothing using count of the tuple, alpha, 
            # count of previous tag, alpha, and number of total tags
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)

    ### END CODE HERE ###
    
    return A

In [25]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000006491
A at row 3, col 1: 0.0003
View a subset of transition matrix A
          PINDFS    PINDMP    PINDMS    PINTFS    PPER1S
PINDFS  0.000008  0.000008  0.000008  0.000008  0.000008
PINDMP  0.000012  0.000012  0.000012  0.000012  0.000012
PINDMS  0.000001  0.000001  0.000001  0.000001  0.000001
PINTFS  0.000484  0.000484  0.000484  0.000484  0.000484
PPER1S  0.000002  0.000002  0.000002  0.000002  0.000002


- la matrice d'emission regroupe les proba des mots sachant le tags.
- les colonnes sont representes par les mots et les lignes sont representes par les tags.

In [26]:
# GRADED FUNCTION: create_emission_matrix

def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Input: 
        alpha: tuning parameter used in smoothing 
        tag_counts: a dictionary mapping each tag to its respective count
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        B: a matrix of dimension (num_tags, len(vocab))
    '''
    
    # get the number of POS tag
    num_tags = len(tag_counts)
    
    # Get a list of all POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Get the total number of unique words in the vocabulary
    num_words = len(vocab)
    
    # Initialize the emission matrix B with places for
    # tags in the rows and words in the columns
    B = np.zeros((num_tags, num_words))
    
    # Get a set of all (POS, word) tuples 
    # from the keys of the emission_counts dictionary
    emis_keys = set(list(emission_counts.keys()))
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    
    # Go through each row (POS tags)
    for i in range(num_tags): # complete this line
        
        # Go through each column (words)
        for j in range(num_words): # complete this line

            # Initialize the emission count for the (POS tag, word) to zero
            count = 0
                    
            # Define the (POS tag, word) tuple for this row and column
            key =  (all_tags[i],vocab[j])

            # check if the (POS tag, word) tuple exists as a key in emission counts
            if key in emission_counts.keys(): # complete this line
        
                # Get the count of (POS tag, word) from the emission_counts d
                count = emission_counts[key]
                
            # Get the count of the POS tag
            count_tag = tag_counts[all_tags[i]]
                
            # Apply smoothing and store the smoothed value 
            # into the emission matrix B for this row and column
            B[i,j] = (count + alpha) / (count_tag+ alpha*num_words)

    ### END CODE HERE ###
    return B

In [27]:
# creating your emission probability matrix. this takes a few minutes to run. 
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

# Try viewing emissions for a few words in a sample dataframe
cidx  = ['1','médecins', 'et', 'scientifiques', 'fous']

# Get the integer ID for each word
cols = [vocab[a] for a in cidx]

# Choose POS tags to show in a sample dataframe
rvals =['ADJ', 'ADJMP', 'ADJMS', 'ADV', 'AUX', 'CHIF',]

# For each POS tag, get the row number from the 'states' list
rows = [states.index(a) for a in rvals]

# Get the emissions for the sample of words, and the sample of POS tags
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000005093
View Matrix position at row 3, column 1: 0.000000140
                  1      médecins            et  scientifiques          fous
ADJ    1.005692e-06  1.005692e-06  1.005692e-06   1.005692e-06  1.005692e-06
ADJMP  3.398655e-07  3.398655e-07  3.398655e-07   3.402054e-04  3.398655e-07
ADJMS  1.358633e-07  1.358633e-07  1.358633e-07   1.358633e-07  1.358633e-07
ADV    7.575562e-08  7.575562e-08  7.575562e-08   7.575562e-08  7.575562e-08
AUX    8.734128e-08  8.734128e-08  8.734128e-08   8.734128e-08  8.734128e-08
CHIF   9.857355e-03  1.095249e-07  1.095249e-07   1.095249e-07  1.095249e-07


# Viterbi Algorithm and Dynamic Programming

In [28]:
# GRADED FUNCTION: initialize
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Input: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
    # Get the total number of unique POS tags
    num_tags = len(tag_counts)
    
    # Initialize best_probs matrix 
    # POS tags in the rows, number of words in the corpus as the columns
    best_probs = np.zeros((num_tags, len(corpus)))
    
    # Initialize best_paths matrix
    # POS tags in the rows, number of words in the corpus as columns
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    # Define the start token
    s_idx = states.index("--s--")
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    
    # Go through each of the POS tags
    for i in range(num_tags): # complete this line
        
        # Handle the special case when the transition from start token to POS tag i is zero
        if A[s_idx,i] == 0: # complete this line
            
            # Initialize best_probs at POS tag 'i', column 0, to negative infinity
            best_probs[i,0] = float('-inf')
        
        # For all other cases when transition from start token to POS tag i is non-zero:
        else:
            
            # Initialize best_probs at POS tag 'i', column 0
            # Check the formula in the instructions above
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i,vocab[corpus[0]]] )
                        
    ### END CODE HERE ### 
    return best_probs, best_paths

In [29]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [30]:
# Test the function
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -24.1327
best_paths[2,3]: 0.0000


In [31]:
# GRADED FUNCTION: viterbi_forward
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    # Get the number of unique POS tags (which is the num of rows in best_probs)
    num_tags = best_probs.shape[0]
    
    # Go through every word in the corpus starting from word 1
    # Recall that word 0 was initialized in `initialize()`
    for i in range(1, len(test_corpus)): 
        
        # Print number of words processed, every 5000 words
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
            
        ### START CODE HERE (Replace instances of 'None' with your code EXCEPT the first 'best_path_i = None') ###
        # For each unique POS tag that the current word can be
        for j in range(num_tags): # complete this line
            
            # Initialize best_prob for word i to negative infinity
            best_prob_i = float("-inf")
            
            # Initialize best_path for current word i to None
            best_path_i = None

            # For each POS tag that the previous word can be:
            for k in range(num_tags): # complete this line
            
                # Calculate the probability = 
                # best probs of POS tag k, previous word i-1 + 
                # log(prob of transition from POS k to POS j) + 
                # log(prob that emission of POS j is word i)
                prob = best_probs[k,i-1]+math.log(A[k,j]) +math.log(B[j,vocab[test_corpus[i]]])

                # check if this path's probability is greater than
                # the best probability up to and before this point
                if prob > best_prob_i: # complete this line
                    
                    # Keep track of the best probability
                    best_prob_i = prob
                    
                    # keep track of the POS tag of the previous word
                    # that is part of the best path.  
                    # Save the index (integer) associated with 
                    # that previous word's POS tag
                    best_path_i = k

            # Save the best probability for the 
            # given current word's POS tag
            # and the position of the current word inside the corpus
            best_probs[j,i] = best_prob_i
            
            # Save the unique integer ID of the previous POS tag
            # into best_paths matrix, for the POS tag of the current word
            # and the position of the current word inside the corpus.
            best_paths[j,i] = best_path_i

        ### END CODE HERE ###
    return best_probs, best_paths

In [32]:

best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000


In [33]:
# Test this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -33.9225
best_probs[0,4]: -55.6327


In [34]:
# UNQ_C7 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_backward
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    # Get the number of words in the corpus
    # which is also the number of columns in best_probs, best_paths
    m = best_paths.shape[1] 
    
    # Initialize array z, same length as the corpus
    z = [None] * m
    
    # Get the number of unique POS tags
    num_tags = best_probs.shape[0]
    
    # Initialize the best probability for the last word
    best_prob_for_last_word = float('-inf')
    
    # Initialize pred array, same length as corpus
    pred = [None] * m
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    ## Step 1 ##
    
    # Go through each POS tag for the last word (last column of best_probs)
    # in order to find the row (POS tag integer ID) 
    # with highest probability for the last word
    for k in range(num_tags): # complete this line

        # If the probability of POS tag at row k 
        # is better than the previosly best probability for the last word:
        if best_probs[k,-1]>best_prob_for_last_word: # complete this line
            
            # Store the new best probability for the lsat word
            best_prob_for_last_word = best_probs[k,-1]
    
            # Store the unique integer ID of the POS tag
            # which is also the row number in best_probs
            z[m - 1] = k
            
    # Convert the last word's predicted POS tag
    # from its unique integer ID into the string representation
    # using the 'states' dictionary
    # store this in the 'pred' array for the last word
    pred[m - 1] = states[k]
    
    ## Step 2 ##
    # Find the best POS tags by walking backward through the best_paths
    # From the last word in the corpus to the 0th word in the corpus
    for i in range(len(corpus)-1, -1, -1): # complete this line
        
        # Retrieve the unique integer ID of
        # the POS tag for the word at position 'i' in the corpus
        pos_tag_for_word_i = best_paths[np.argmax(best_probs[:,i]),i]
        
        # In best_paths, go to the row representing the POS tag of word i
        # and the column representing the word's position in the corpus
        # to retrieve the predicted POS for the word at position i-1 in the corpus
        z[i - 1] = best_paths[pos_tag_for_word_i,i]
        
        # Get the previous word's POS tag in string form
        # Use the 'states' dictionary, 
        # where the key is the unique integer ID of the POS tag,
        # and the value is the string representation of that POS tag
        pred[i - 1] = states[pos_tag_for_word_i]
        
     ### END CODE HERE ###
    return pred

In [35]:
# Run and test your function
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['Nancy', ',', 'est', 'une', '--unk--', 'française'] 
 ['PROPN', 'PUNCT', 'AUX', 'DINTFS', 'NFS', 'ADJFS'] 

The prediction for pred[0:8] is: 
 ['PPER1S', 'VERB', 'COSUB', 'PREP', 'PDEMMS', 'COCO', 'DET'] 
 ['Je', 'sens', "qu'", 'entre', 'ça', 'et', 'les']


In [36]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The third word is: entre
Your prediction is: PREP
Your corresponding label y is:  entre	PREP



In [37]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: compute_accuracy
def compute_accuracy(pred, y):
    '''
    Input: 
        pred: a list of the predicted parts-of-speech 
        y: a list of lines where each word is separated by a '\t' (i.e. word \t tag)
    Output: 
        
    '''
    num_correct = 0
    total = 0
    
    # Zip together the prediction and the labels
    for prediction, y in zip(pred, y):
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # Split the label into the word and the POS tag
        word_tag_tuple = y.split()
        
        # Check that there is actually a word and a tag
        # no more and no less than 2 items
        if len(word_tag_tuple)!=2: # complete this line
            continue 

        # store the word and tag separately
        word, tag = word_tag_tuple
        
        # Check if the POS tag label matches the prediction
        if prediction == tag: # complete this line
            
            # count the number of times that the prediction
            # and label match
            num_correct += 1
            
        # keep track of the total number of examples (that have valid labels)
        total += 1
        
        ### END CODE HERE ###
    return num_correct/total

In [38]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.9168


In [39]:
print('The 5 word is:', prep[5])
print('Your prediction is:', pred[5])
print('Your corresponding label y is: ', y[5])

The 5 word is: et
Your prediction is: COCO
Your corresponding label y is:  et	COCO

